<center><h2>Kaggle Data Science Project:</h2></center>
<center><h1>[Insert name]</h1></center>

<img src="https://www.dataapplab.com/wp-content/uploads/2016/10/kaggle-logo-transparent-300.png" style="width: 30rem"/>

## Table of contents:
0. [Prerquisites;](#prerequisites)
1. [Acquisition;](#acquisition)
2. [Exploration & Preprocessing;](#exploration-preprocessing)
3. [Analysis;](#analysis)
4. [Interpretation;](#interpretation)

## To do:
- 
-

<br>

## <a id="prerequisites">1. Prerequisites:</a>

In [ ]:
# Import data science tools:

!rm -rf ./cornelia
!git clone -b cornelia https://github.com/augustinasn/_data_science_projects.git ./cornelia
!rm -rf ./cornelia/.rf

In [ ]:
# Packages:

from cornelia.imports import *
from cornelia.helpers import display_opts
from cornelia.extraction import read_feather, pickle_obj, unpickle_obj
from cornelia.preprocessing import match_cols, drop_cols, fill_NAs, one_hot_encode, category_encode, split_df, drop_NAs
from cornelia.visualization import print_missing_data, print_descriptive_stats, print_categories, plot_data, print_df, plot_distributions, pretty_print_dict
from cornelia.analysis import score_regr, score_class, feature_importance, confidence, similiar_features, score_with_cols_dropped, pdps, Pipeline

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier


import json
import itertools
import datetime

# Options:

%matplotlib inline
%load_ext autoreload
%autoreload 2

display_opts(decimal_numbers=2,
             max_rows=1000,
             max_cols=30)

## <a id="acquisition">1. Acquisition</a>

In [ ]:
# Load raw data:

raw_train_df = pd.read_csv("")
raw_test_df = pd.read_csv("")

## <a id="exploration-preprocessing">2. Exploration and Preprocessing:</a>

In [ ]:
# Match columns:

raw_train_df = match_cols(df1=raw_train_df,
                          df2=raw_test_df,
                          omit=["y"])

In [ ]:
# Observe missing data:

print_missing_data(dfs=[raw_train_df, raw_test_df],
                   labels=["train", "test"])

In [ ]:
# Descriptive stats:

print_descriptive_stats(dfs=[raw_train_df, raw_test_df],
                        labels=["train", "test"])

In [ ]:
# Distributions

plot_distributions(df=raw_train_df,
                   target="y",
                   n_bins=5)

In [ ]:
# Print histograms for each feature:
    
plot_data(dfs=[raw_train_df, raw_test_df],
          labels=["train", "test"],
          bench=1,
          n_bins=4)

In [ ]:
# Backup data:

pickle_obj([raw_train_df, raw_test_df], "train_test_dfs")

## <a id="analysis">3. Analysis & Interpretation:</a>

In [ ]:
# Unbackup data:

raw_train_df, raw_test_df = unpickle_obj("train_test_dfs")

### 3.1. Iteration #1 - test dropping NaNs on each axis:

In [ ]:
config_1 = {"sample": 0.2,
            "nas": {"mode": "drop",
                    "params": {"axis": [0, 1]}},
            "categoricals": {"mode": "category_encode"},
            "split": 0.5,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": 40}
                           }
           }

p = Pipeline(config=config_1, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y")

p.run()

### 3.2. Iteration #2 - test filling NaNs with different parameters:

In [ ]:
config_2 = {"sample": 0.2,
            "nas": {"mode": "fill",
                    "params": {"num_method": ["median", "mean", None],
                               "cat_method": ["mode", None],
                               "was_missing": [True, False]}},
            "categoricals": {"mode": "category_encode"},
            "split": 0.5,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": 40}
                           }
           }

p = Pipeline(config=config_2, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y")

p.run()

In [ ]:
# Print score history:

p.scores()

### 3.3. Iteration #3 - testing OneHotEncoding & feature importance:

In [ ]:
config_3 = {"sample": 0.2,
            "nas": {"mode": "fill",
                    "params": {"num_method": "median",
                               "cat_method": "mode",
                               "was_missing": True}},
            "categoricals": {"mode": "one_hot_encode",
                             "params": {"card_threshold": 15}},
            "split": 0.5,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": 40}
                           }
           }

p = Pipeline(config=config_3, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y",
             backup=True)

p.run()

In [ ]:
# Feature importance:

fi_cols_to_drop = p.feature_importance()

# Backup feature importance result:

pickle_obj(fi_cols_to_drop, "iteration_3_fi_cols_to_drop")

### 3.4. Iteration #4 - dropping unimportant features + testing colinear features:

In [ ]:
fi_cols_to_drop = unpickle_obj("fi_cols_to_drop")

config_4 = {"sample": 0.2,
            "nas": {"mode": "fill",
                    "params": {"num_method": "median",
                               "cat_method": "mode",
                               "was_missing": True}},
            "categoricals": {"mode": "category_encode"},
            "drop_cols": fi_cols_to_drop,
            "split": 0.5,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": 40}
                           }
           }

p = Pipeline(config=config_2, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y")

p.run()

In [ ]:
# Find colinear features:

p.colinear_features()

In [ ]:
# Check if dropping any of colinear feature will impact score:

potentially_colinear_features = [["yearly_income=>was_missing", "credit_score=>was_missing"],
                                 ["max_open_credit", "credit_balance"],
                                 ["monthly_debt", "yearly_income"]]

# Backup feature importance result:

pickle_obj(potentially_colinear_features, "iteration_4_potentially_colinear_features")

### 3.5. Iteration #5 - test dropping different combinations of colinear features:

In [ ]:
fi_cols_to_drop = unpickle_obj("fi_cols_to_drop")
potentially_colinear_features = unpickle_obj("model_4_potentially_colinear_features")
feature_pairs_drop = list(itertools.product(*[t + [None] for t in potentially_colinear_features]))


config_5 = {"sample": 0.2,
            "nas": {"mode": "fill",
                    "params": {"num_method": "median",
                               "cat_method": "mode",
                               "was_missing": True}},
            "categoricals": {"mode": "category_encode"},
            "drop_cols": fi_cols_to_drop + feature_pairs_drop,
            "split": 0.5,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": 40}
                           }
           }

p = Pipeline(config=config_5, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y")

p.run()

In [ ]:
colinear_features = ["yearly_income=>was_missing", "max_open_credit", "monthly_debt"]
pickle_obj(colinear_features, "iteration_5_colinear_features")

### 3.6. Iteration #6 - dropping colinear features and testing different splits:

In [ ]:
fi_cols_to_drop = unpickle_obj("fi_cols_to_drop")
colinear_features = unpickle_obj("iteration_5_colinear_features")

config_6 = {"sample": 0.2,
            "nas": {"mode": "fill",
                    "params": {"num_method": "median",
                               "cat_method": "mode",
                               "was_missing": True}},
            "categoricals": {"mode": "category_encode"},
            "drop_cols": fi_cols_to_drop + feature_pairs_drop,
            "split": 0.9,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": 40}
                           }
           }

p = Pipeline(config=config_6, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y")

p.run()

### 3.7. Iteration #7 - RandomForrest parameter tuning:

In [ ]:
fi_cols_to_drop = unpickle_obj("fi_cols_to_drop")
colinear_features = unpickle_obj("iteration_5_colinear_features")

config_7 = {"sample": 0.2,
            "nas": {"mode": "fill",
                    "params": {"num_method": ["median", "mean", None],
                               "cat_method": ["mode", None],
                               "was_missing": [True, False]}},
            "categoricals": {"mode": "category_encode"},
            "drop_cols": fi_cols_to_drop + feature_pairs_drop,
            "split": 0.9,
            "prediction": {"model": "rf",
                           "params": {"n_estimators": [50, 75, 100, 150],
                                      "max_samples": [50_000, 100_000, 250_000],
                                      "max_features": [0.7, 0.8, 0.9],
                                      "min_samples_leaf": [1, 3, 5]}
                           }
           }

p = Pipeline(config=config_7, 
             train_df=raw_train_df,
             test_df=raw_test_df,
             target="y")

p.run()

### 4. Interpretation:

In [ ]:
# Confidence of predictions for each feature and it's each category:

confidence(model=m,
           df=valid_X,
           n_bins=5)

In [ ]:
# Actual relationship between independant and dependant features (partial dependance):

pdps(model=m,
          data=train_X.sample(10_000),
          target="y",
          omit=fi_cols_to_drop + colin_cols_to_drop,
          clusters=5)
